In [1]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eric_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np 
import pandas as pd

from pprint import pprint

In [3]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [5]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [6]:
#nltk stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from','subject','re','edu','use'])

In [7]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:\\Users\\eric_\\Downloads\\My Project-499-f28c2476b9b8.json"

In [8]:
from google.cloud import bigquery
client = bigquery.Client()

In [9]:
sql = '''
        SELECT title
        FROM `bigquery-public-data.stackoverflow.tags` FULL OUTER Join `bigquery-public-data.stackoverflow.posts_questions`
        ON tag_name = tags
        where tags like'%flutter%' and score > 3
        order by score desc
'''
query_job = client.query(
    sql,
    location="US",
)
flutter = []
for row in query_job:
    flutter += row
#print(flutter)


In [10]:
#tokenize words 
def tokenize_words(txt):
    yield(gensim.utils.simple_preprocess(str(txt), deacc=True)) #deacc removes punctuations

data_words = list(tokenize_words(flutter))
#print(data_words)

In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized)

In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])

In [15]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('abi', 1),
  ('able', 2),
  ('abnormally', 1),
  ('absolute', 2),
  ('accept', 3),
  ('access', 14),
  ('accessibility', 1),
  ('accidental', 1),
  ('accord', 2),
  ('account', 1),
  ('achieve', 1),
  ('action', 5),
  ('activity', 4),
  ('actually', 1),
  ('ad', 4),
  ('add', 67),
  ('additional', 1),
  ('address', 1),
  ('adjust', 2),
  ('agreement', 1),
  ('alarm', 1),
  ('alert', 1),
  ('alertdialog', 2),
  ('align', 5),
  ('allow', 3),
  ('already', 8),
  ('also', 2),
  ('alternative', 2),
  ('always', 3),
  ('amount', 3),
  ('analytic', 1),
  ('android', 32),
  ('android_studio', 9),
  ('androidx', 2),
  ('angulardart', 1),
  ('animate', 9),
  ('animation', 13),
  ('annotation', 3),
  ('anonymous', 1),
  ('anywhere', 1),
  ('api', 1),
  ('apiexception', 1),
  ('apk', 1),
  ('app', 26),
  ('appbar', 1),
  ('appbardesign', 1),
  ('appear', 6),
  ('appearance', 1),
  ('append', 1),
  ('apple', 1),
  ('application', 22),
  ('apply', 4),
  ('arabic', 1),
  ('arbitrary', 2),
  ('arch

In [16]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics= 3, 
                                           random_state=20,
                                           update_every=1,
                                           chunksize=500,
                                           passes=15,
                                           alpha=15,
                                           per_word_topics=True)

In [17]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.001*"flutter" + 0.001*"widget" + 0.001*"use" + 0.001*"image" + '
  '0.001*"add" + 0.001*"error" + 0.001*"screen" + 0.001*"create" + '
  '0.001*"change" + 0.001*"text"'),
 (1,
  '0.005*"flutter" + 0.002*"widget" + 0.002*"use" + 0.002*"screen" + '
  '0.002*"image" + 0.001*"change" + 0.001*"error" + 0.001*"add" + 0.001*"text" '
  '+ 0.001*"color"'),
 (2,
  '0.059*"flutter" + 0.023*"widget" + 0.017*"use" + 0.014*"image" + '
  '0.013*"text" + 0.012*"error" + 0.011*"screen" + 0.011*"change" + '
  '0.010*"add" + 0.010*"get"')]


In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.456523018963965

Coherence Score:  0.21147347686712922


In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\eric_\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
2      0.097753 -0.0       1        1  99.415817
1     -0.039182 -0.0       2        1   0.295638
0     -0.058571 -0.0       3        1   0.288543, topic_info=     Category        Freq     Term       Total  loglift  logprob
458   Default  360.000000  flutter  360.000000  30.0000  30.0000
1251  Default  137.000000   widget  137.000000  29.0000  29.0000
1212  Default  104.000000      use  104.000000  28.0000  28.0000
986   Default   69.000000   screen   69.000000  27.0000  27.0000
551   Default   84.000000    image   84.000000  26.0000  26.0000
...       ...         ...      ...         ...      ...      ...
43     Topic3    0.014525      app   24.529137  -1.5836  -7.0995
143    Topic3    0.014562     call   31.998575  -1.8470  -7.0970
31     Topic3    0.014552  android   30.090200  -1.7862  -7.0977
973    Topic3    0.014507      run   23.592958  -1.5460  -7.1008
1037   Topic3    0.014515     size   29.203848  -1.7588  -7.1002

[204 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
15        1  1.005008            add
31        1  0.997002        android
34        1  0.790726    angulardart
43        1  0.978428            app
71        1  0.791009   autocomplete
...     ...       ...            ...
1219      1  0.790562  valuenotifier
1242      1  0.980935            way
1251      1  0.999655         widget
1256      1  0.790681           word
1257      1  0.994227           work

[99 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])